In [145]:
%load_ext autoreload
%autoreload 2

import time

import numpy as np
import pandas as pd
import requests
import requests_cache
import rpy2.robjects as ro
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
from smiles2graph import getDrugGraph
from tqdm import tqdm
from utils import (get_canonical_smiles, get_pubchem_info, get_smiles_from_cid,
                   get_smiles_from_nsc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
drugs = torch.load("../data/drug_list.pt", weights_only=False)
len(drugs)

5734

In [147]:
drugs

[1,
 186,
 295,
 577,
 721,
 740,
 742,
 750,
 752,
 755,
 757,
 762,
 844,
 1026,
 1027,
 1390,
 1895,
 1906,
 2013,
 2183,
 2384,
 2654,
 3039,
 3051,
 3053,
 3088,
 3364,
 3590,
 3894,
 3905,
 3970,
 4320,
 4728,
 5031,
 5112,
 5159,
 5889,
 5890,
 6171,
 6175,
 6268,
 6396,
 6814,
 7365,
 7521,
 7525,
 7534,
 8101,
 8117,
 8120,
 8423,
 8451,
 8806,
 9168,
 9226,
 9363,
 9364,
 9665,
 9856,
 9908,
 9948,
 11779,
 11969,
 12825,
 13119,
 13502,
 13875,
 14666,
 14974,
 14975,
 15200,
 15309,
 15384,
 15780,
 18268,
 18298,
 18334,
 18335,
 18894,
 19492,
 19893,
 19912,
 19994,
 20256,
 21075,
 21205,
 21206,
 21548,
 21729,
 22842,
 23216,
 23615,
 23759,
 24500,
 24559,
 24817,
 24818,
 24819,
 25154,
 25953,
 26271,
 26980,
 27592,
 27640,
 28130,
 30084,
 30706,
 30897,
 30986,
 32065,
 32743,
 32944,
 32946,
 33092,
 33410,
 33434,
 33530,
 33669,
 33832,
 34462,
 34931,
 35542,
 36354,
 36390,
 36508,
 37364,
 38270,
 38721,
 38876,
 39202,
 41458,
 43486,
 43675,
 45383,
 453

In [90]:
cid = pd.read_csv(
    "~/Downloads/nsc_sid_cid.csv",
    on_bad_lines="skip",
    usecols=["NSC", "CID"],
)
cid = cid.dropna().reset_index(drop=True)
cid = cid[cid["NSC"].isin(drugs)]

In [91]:
j = pd.read_csv("~/Downloads/nsc_smiles.csv").dropna()
j

,NSC,SMILES
0,1,CC1=CC(=O)C=CC1=O
1,2,S(Sc1nc2ccccc2s1)c3nc4ccccc4s3
2,3,Oc1c(Cl)cc(cc1[N+](=O)[O-])[N+](=O)[O-]
3,4,Nc1ncc(s1)[N+](=O)[O-]
4,5,Nc1ccc2C(=O)c3ccccc3C(=O)c2c1
...,...,...
321745,853988,CN1CCN(C[C@@H]2COc3ccc(OCc4ccnc(n4)c5ccc(OC[C@...
321746,854009,C[C@H]1CN(C)CCN1c2cc(NC(=O)Cn3cc(c4cc(C(=O)N)c...
321747,854010,CCNCCc1ccc(CN(CC)c2cc(OC)ccc2[C@@H]3CCc4cc(O)c...
321748,854043,O.CCn1c(c2CC(C)(C)COC(=O)[C@@H]3CCCN(N3)C(=O)[...


In [92]:
drugs

[1895,
 2654,
 3051,
 8117,
 15200,
 15384,
 23216,
 33832,
 36390,
 41458,
 46401,
 51143,
 53398,
 61586,
 63345,
 63445,
 64948,
 69856,
 71901,
 71935,
 71936,
 72847,
 75603,
 76779,
 77471,
 77950,
 83340,
 83628,
 85775,
 91768,
 93819,
 94217,
 94219,
 95580,
 96019,
 99282,
 100045,
 100844,
 102627,
 105023,
 105024,
 107041,
 109229,
 109441,
 110326,
 111180,
 112758,
 114573,
 117212,
 118742,
 122716,
 124668,
 126730,
 132319,
 139490,
 141520,
 142982,
 143020,
 145168,
 146443,
 149501,
 149789,
 150817,
 157365,
 159228,
 168411,
 169534,
 170105,
 174120,
 174280,
 180512,
 191959,
 200692,
 210717,
 224131,
 227185,
 227228,
 237020,
 294601,
 295564,
 298223,
 305264,
 307241,
 308312,
 311148,
 325659,
 325660,
 325661,
 327181,
 349438,
 355638,
 356232,
 356235,
 363787,
 372160,
 372162,
 378842,
 378843,
 378895,
 378896,
 381811,
 381867,
 406472,
 525661,
 526598,
 600391,
 600392,
 603539,
 608604,
 608984,
 613678,
 614487,
 615284,
 615291,
 618077,
 6182

In [93]:
l = pd.read_csv(
    "~/Downloads/4119094074703879483.txt.gz", sep="\t", header=None
).dropna()
l.columns = ["NSC", "CID"]
l

,NSC,CID
0,1895,15078.0
1,2654,6597.0
2,3051,31254.0
3,8117,54676860.0
4,15200,6711183.0
...,...,...
405,719344,2187.0
425,744479,20189670.0
426,747803,23245656.0
427,747804,54612711.0


In [94]:
k = pd.concat([cid, l]).dropna()
k = k.drop_duplicates()
k["SMILES"] = [get_smiles_from_cid(str(int(i))) for i in k["CID"]]

Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...


In [95]:
k

,NSC,CID,SMILES
1874,1895,15078.0,C1(=NC(=NN1)N)N
2627,2654,6597.0,C(C(=O)O)(Cl)Cl
3014,3051,31254.0,CNC=O
7934,8117,54676860.0,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,6711183.0,[N+](=O)(O)[O-].[Ga]
...,...,...,...
254710,719344,2187.0,CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N
265083,744479,20189670.0,C(C(=O)O)(Cl)Cl.[Na+]
266264,747803,23245656.0,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)C(=O)O...
266265,747804,54612711.0,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)O)C(=O...


In [96]:
fromCID = k[k.dropna().SMILES != "'PUGREST.ServerBusy'"][["NSC", "SMILES"]]
fromCID

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
254710,719344,CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N
265083,744479,C(C(=O)O)(Cl)Cl.[Na+]
266264,747803,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)C(=O)O...
266265,747804,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)O)C(=O...


In [97]:
fromCID.sort_values("NSC")

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
254710,719344,CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N
265083,744479,C(C(=O)O)(Cl)Cl.[Na+]
266264,747803,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)C(=O)O...
266265,747804,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)O)C(=O...


In [98]:
cid = pd.read_csv(
    "~/Downloads/nsc_sid_cid.csv",
    on_bad_lines="skip",
    usecols=["NSC", "CID"],
)
cid = cid.dropna().reset_index(drop=True)
cid = cid[cid["NSC"].isin(drugs)]
cid["SMILES"] = [get_smiles_from_cid(str(int(i))) for i in cid["CID"]]
cid = cid[["NSC", "SMILES"]]

Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...
Server is busy. Retrying in 5 seconds...


In [99]:
o = pd.read_csv("~/Downloads/nsc_smiles.csv")
o = o[o.NSC.isin(drugs)].dropna()

In [100]:
cellminer = importr("rcellminer")
r_nsc_set = StrVector([str(i) for i in drugs])
r_smiles = cellminer.getSmiles(r_nsc_set)
r_smiles = pd.DataFrame([drugs, list(r_smiles)]).T
r_smiles = r_smiles[r_smiles[1] != ro.NA_Character]
r_smiles = r_smiles[r_smiles[1] != ""]
r_smiles.columns = ["NSC", "SMILES"]

In [101]:
k = pd.read_csv(
    "~/Downloads/2717352336174282052.txt.gz", sep="\t", header=None
).dropna()
k.columns = ["NSC", "SMILES"]
k

,NSC,SMILES
0,1895,C1(=NC(=NN1)N)N
1,2654,C(C(=O)O)(Cl)Cl
2,3051,CNC=O
3,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
4,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
405,719344,CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N
425,744479,C(C(=O)O)(Cl)Cl.[Na+]
426,747803,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)C(=O)O...
427,747804,C1=CC=C2C(=C1)C(=O)/C(=C\NC3=CC(=C(C=C3)O)C(=O...


In [102]:
smiles = pd.concat([cid, o, r_smiles, k]).drop_duplicates(subset="NSC")
smiles

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
225661,633243,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(Cc4cccc(n4...
226296,634169,[R]Oc1ccc(cc1)C(=O)c2cccc(n2)C(=O)c3ccc(OC4CCC...
226297,634170,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(cc3)C(C)(C...
239813,661964,CCC(C)(C)c1ccc(O)c([R]SSc2cc(cc(c2O)[RH])C(C)(...


In [103]:
smiles

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
225661,633243,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(Cc4cccc(n4...
226296,634169,[R]Oc1ccc(cc1)C(=O)c2cccc(n2)C(=O)c3ccc(OC4CCC...
226297,634170,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(cc3)C(C)(C...
239813,661964,CCC(C)(C)c1ccc(O)c([R]SSc2cc(cc(c2O)[RH])C(C)(...


In [104]:
df = pd.concat([smiles, fromCID]).drop_duplicates()
df

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
225661,633243,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(Cc4cccc(n4...
226296,634169,[R]Oc1ccc(cc1)C(=O)c2cccc(n2)C(=O)c3ccc(OC4CCC...
226297,634170,[R]Oc1ccc(cc1)S(=O)(=O)c2ccc(Oc3ccc(cc3)C(C)(C...
239813,661964,CCC(C)(C)c1ccc(O)c([R]SSc2cc(cc(c2O)[RH])C(C)(...


In [57]:
missing = pd.DataFrame(sorted(set(drugs) - set(df.NSC)))
# .to_csv('missing.csv')

In [59]:
missing[0]

0       15384
1       36390
2       46401
3       53398
4       63445
        ...  
249    807448
250    811189
251    819151
252    819152
253    819153
Name: 0, Length: 254, dtype: int64

In [68]:
l = pd.read_excel("~/Downloads/nsc_chemcal_name.xltx")
l

,NSC,NAME
0,1,WLN: L6V DVJ B1
1,1,"2,5-CYCLOHEXADIENE-1,4-DIONE, 2-METHYL- (9CI)"
2,1,P-TOLUQUINONE
3,1,TOLUQUINONE (VAN)
4,1,TOLYLQUINONE
...,...,...
265409,854044,RMC7977
265410,854044,RMC-7977 HYDRATE
265411,854044,RMC-7977
265412,854044,CT-RMC7977


In [71]:
name = l[l.NSC.isin(missing[0])]
name

,NSC,NAME
59014,15384,"SULFONATED CASTOR OIL, SODIUM SALT"
59015,15384,RICINUS COMMUNIS
86013,36390,"RESIN PODOPHYLLUM, USP, FROM MANDRAKE ROOTS, C..."
86014,36390,PODOPHYLLUM (THE RESIN)
86015,36390,PODOPHYLLIN
...,...,...
259563,807448,PEGYLATED CYSTATHIONINE BETA-SYNTHASE (CBS) EN...
260265,811189,HUMAN SERUM ALBUMIN IS DOMINANT STRUCTURE
262000,819151,THHHKHPGG[6-AMINOHEXANOIC ACID]GRKKRRQRRRPPQ
262001,819152,TEDSKTHHH[6-AMINOHEXANOIC ACID]GRKKRRQRRRPPQ


In [80]:
name["SMILES"] = [get_canonical_smiles(i) for i in name.NAME]
name

/var/folders/s4/6xgylm7x3yzgfsk6sbvk_7_c0000gn/T/ipykernel_8125/1265607948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name['SMILES'] = m


,NSC,NAME,SMILES
59014,15384,"SULFONATED CASTOR OIL, SODIUM SALT",None
59015,15384,RICINUS COMMUNIS,None
86013,36390,"RESIN PODOPHYLLUM, USP, FROM MANDRAKE ROOTS, C...",None
86014,36390,PODOPHYLLUM (THE RESIN),None
86015,36390,PODOPHYLLIN,COC1=CC(=CC(=C1OC)OC)C2C3C(COC3=O)C(C4=CC5=C(C...
...,...,...,...
259563,807448,PEGYLATED CYSTATHIONINE BETA-SYNTHASE (CBS) EN...,None
260265,811189,HUMAN SERUM ALBUMIN IS DOMINANT STRUCTURE,None
262000,819151,THHHKHPGG[6-AMINOHEXANOIC ACID]GRKKRRQRRRPPQ,None
262001,819152,TEDSKTHHH[6-AMINOHEXANOIC ACID]GRKKRRQRRRPPQ,None


In [108]:
df = pd.concat([df, name[["NSC", "SMILES"]]]).dropna()
df

,NSC,SMILES
1874,1895,C1(=NC(=NN1)N)N
2627,2654,C(C(=O)O)(Cl)Cl
3014,3051,CNC=O
7934,8117,C(C(C1C(=C(C(=O)O1)O)O)O)O
14565,15200,[N+](=O)(O)[O-].[Ga]
...,...,...
225257,627997,C1=CC=C(C=C1)CC(=O)NC(CCC(=O)N)C(=O)[O-].C1=CC...
225890,632307,CC1(C(=C)N(C2=CC=CC=C21)CCCCCC(=O)O)C
225891,632307,CCCCCC=CCC=CCC=CCC=CCC=CCCC(=O)OC
226967,638719,CCC(C)C1C(CC(=O)OC(C(=O)C(C(=O)NC(C(=O)N2CCCC2...


In [121]:
still = name[name["SMILES"].isna()][["NSC", "SMILES"]].drop_duplicates()
still
# .to_csv('missed.csv')

,NSC,SMILES
59014,15384,None
86013,36390,None
97896,46401,None
106514,53398,None
117094,63445,None
...,...,...
259563,807448,None
260265,811189,None
262000,819151,None
262001,819152,None


In [127]:
query = pd.read_csv("~/Downloads/drug_name_query.csv").dropna()
query = query[query["NSC #"].isin(still["NSC"])]
query["SMILES"] = [get_canonical_smiles(i) for i in query["Drug name"]]
query

An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Ricinus%20communis/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/.alpha.-Sarcin/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/A%208671K119/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/narangomycin/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/corpormon/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/statolon%20(usan)/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/HAMYCIN/cids/JSON
A

An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/N-Cys%20HIV%20(583-599)/cids/JSON
Server is busy. Retrying in 5 seconds...
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Guanidinium%20salt%20of%20Tungstoantimonate/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Peptide%20isolated%20from%20Viscum%20alblum%20extract%20(Iscador)/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Crotoxin%20CD/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/LAM%201C3/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/LAM%202E3/cids/JSON
An error occurred

Server is busy. Retrying in 5 seconds...
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/NKLB/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Water%20extract%20from%20Clivia%20Miniata%20Regel/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Efinrin%20leaves%20extract%20A/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/insulin-methotrexate%20conjugate/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Protein%20Toxin%20A%2023%20-%20MW%20approx.%206700/cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Protein%20toxin%20B%2015%

An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Holotransferrin%20complexed%20w/%20cis-%5BCl2(NH3)4RuIII%5DCl,%20MW=.../cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Holotransferrin%20complexed%20w/(ImH)trans-%5B(Im)2Cl4RuIII,%20M.../cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Fusion%20protein%2059%20(FP59)%20plus%20Protective%20Antigen%20(PA)at%20.../cids/JSON
An error occurred: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Doxorubicin-Transferrin%20conjugate%20(Dox/TrF=2.5),~%20Dox:%202.../cids/JSON
An error occurred: 404 Client Error: Not Found for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/NSC%20701656%20incorporated%20in%20micelles,%200.73%25%20(w/w)%20of%20drug./cids/JSON
An

,NSC #,Drug name,SMILES
859,15384.0,Ricinus communis,None
1628,36390.0,Podophyllin,COC1=CC(=CC(=C1OC)OC)C2C3C(COC3=O)C(C4=CC5=C(C...
2021,46401.0,.alpha.-Sarcin,None
2267,53398.0,A 8671K119,None
2548,63445.0,narangomycin,None
...,...,...,...
55830,807448.0,PEGylated cystathionine beta-synthase (CBS) en...,None
56100,811189.0,Human Serum Albumin is dominant structure,None
56791,819151.0,THHHKHPGG[6-aminohexanoic acid]GRKKRRQRRRPPQ,None
56792,819152.0,TEDSKTHHH[6-aminohexanoic acid]GRKKRRQRRRPPQ,None


In [131]:
query = query.dropna()[["NSC #", "SMILES"]]
query.columns = ["NSC", "SMILES"]
query

,NSC,SMILES
1628,36390.0,COC1=CC(=CC(=C1OC)OC)C2C3C(COC3=O)C(C4=CC5=C(C...
2739,69856.0,CC1C(C(C(C(O1)OC2C3CC#CC4C(O4)(C#CC3=CC2OC(=O)...
3394,91768.0,CC1CCC(C(C(CC(C(C=CC=C(C(=O)OC(C(C=CC=C(C(CC(C...
3487,94219.0,CC1C=CC=CC=CC=CC=CC=CC=CC(CC(C(C(CC(=O)CC(CC(C...
3794,105024.0,CC1C(C(CC(O1)OC2CC(OC(C2O)C)OC3=CC4=CC5=C(C(=O...
3919,109229.0,CC1=CC(=C(C=C1)C(=O)C(=C)CN(C)C)C
5472,170105.0,COC1=CC(=C2C(=C1)OC(=C)C(=O)N2)C(=O)OC
8762,608604.0,CCC(C(C)C(C(C)C(C(C)C1C(C2C(O2)C(CCCC(CC(CC3CC...
10661,622493.0,C(C1C2C(C(C(O1)OC3C(OC(C(C3O)O)OC4C(OC(C(C4O)O...
10662,622494.0,C(C1C2C(C(C(O1)OC3C(OC(C(C3O)O)OC4C(OC(C(C4O)O...


In [142]:
df = pd.concat([query, df]).drop_duplicates()
df = pd.concat([df, pd.read_csv("missed.csv").dropna()]).dropna()
df

,NSC,SMILES
1628,36390.0,COC1=CC(=CC(=C1OC)OC)C2C3C(COC3=O)C(C4=CC5=C(C...
2739,69856.0,CC1C(C(C(C(O1)OC2C3CC#CC4C(O4)(C#CC3=CC2OC(=O)...
3394,91768.0,CC1CCC(C(C(CC(C(C=CC=C(C(=O)OC(C(C=CC=C(C(CC(C...
3487,94219.0,CC1C=CC=CC=CC=CC=CC=CC=CC(CC(C(C(CC(=O)CC(CC(C...
3794,105024.0,CC1C(C(CC(O1)OC2CC(OC(C2O)C)OC3=CC4=CC5=C(C(=O...
...,...,...
51,622494,C(C1C2C(C(C(O1)OC3C(OC(C(C3O)O)OC4C(OC(C(C4O)O...
56,624239,CC1=CC(=C(C=C1)C(=O)C(=C)CN(C)C)C
58,625770,C1=CC(=C(C=C1C(=C2C=CC(=O)C(=C2)C(=O)O)C3=CC(=...
62,632307,CCC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC)O)C2=NC5=...


In [144]:
for i in df["NSC"]:
    print(int(i))

36390
69856
91768
94219
105024
109229
170105
608604
622493
622494
625770
1895
2654
3051
8117
15200
23216
33832
41458
51143
63345
83628
85775
93819
95580
96019
102627
107041
109441
111180
112758
117212
118742
132319
139490
141520
142982
143020
145168
146443
149501
150817
159228
169534
174120
174280
200692
210717
224131
227228
294601
298223
307241
349438
363787
372160
381811
406472
526598
615284
615291
618077
618223
618233
618487
618495
618857
619989
620130
623586
624589
625733
626541
626674
627404
627405
633664
633983
633984
634374
634375
638718
640084
643728
646276
653000
653380
656576
656667
658398
661583
666132
666134
666136
666139
666140
666141
666142
666170
666171
666172
666739
666740
668002
668003
668004
668005
668006
668007
668008
668009
668015
668016
668017
668018
668019
668020
670348
670349
670351
670352
670353
670354
670355
670356
670359
670362
670363
670389
670390
670391
670392
670393
670395
673171
673172
674127
678038
678407
678408
678409
678410
678411
678412
683132
683133
6

In [150]:
df.to_csv("../data/hand_gathering_data.csv")

In [179]:
torch.save(list(set(q.NSC)), "../data/tmp.pt")

In [5]:
drug_dict = {}
for i in tqdm(tmp.iterrows()):
    drug_dict[i[1]["NSC"]] = getDrugGraph(i[1]["SMILES"])

174it [00:00, 2623.84it/s]


In [6]:
torch.save(drug_dict, "../data/data_dict.pt")

In [8]:
len(tmp)

174